# Final Project - Super Blocks

In [2]:
#imports
%matplotlib inline

import pandas as pd
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind
import folium
import geocoder
DEFAULT_LOCATION = [32.821932,-117.1509477] #center location to start the map at 
map_sd = folium.Map(location=DEFAULT_LOCATION,zoom_start=10, max_zoom=11, min_zoom=10) #constrain the zoom
map_sd2 = folium.Map(location=DEFAULT_LOCATION,zoom_start=10, max_zoom=11, min_zoom=10) #constrain the zoom
map_sd3 = folium.Map(location=DEFAULT_LOCATION,zoom_start=10, max_zoom=11, min_zoom=10) #constrain the zoom
map_sd4 = folium.Map(location=DEFAULT_LOCATION,zoom_start=10, max_zoom=11, min_zoom=10) #constrain the zoom
map_sd5 = folium.Map(location=DEFAULT_LOCATION,zoom_start=10, max_zoom=11, min_zoom=10) #constrain the zoom
map_sd6 = folium.Map(location=DEFAULT_LOCATION,zoom_start=10, max_zoom=11, min_zoom=10) #constrain the zoom



## Introduction and Background

Although most cities in America are designed in such a way that it is almost necessary to have a car to get around, today, factors like overpopulation, air and noise pollution and lack of free space constantly urge that a more efficient solution is created for people to get around for their daily needs.  Urban neighborhoods today are an intertwined mesh of commercial activity, transports and residences, and at most times, people live close by to the places they usually visit, for which having a car to get around isn’t entirely necessary, especially if there is a more efficient way to get around. However, issues like road safety, pollution, lack of bicycle lanes etc. often cause us to use our cars when there could be no apparent need, which eventually just adds to the problem, hence urging us to recognize a solution that is both more efficient towards the environment and our time, as well as has economic and commercial benefits for society.

Faced with increasing issues of pollution, traffic and crowded spaces, Barcelona’s city planners created and currently implement a system where they blocked off certain urban areas from vehicle transport, and promoted commercial and community use in these areas. The results were astounding: 60% of the total streets in the city freed up space, traffic has reduced overall by 21%, 300km of new cycling lanes have been developed all around the city, commercial activity has boomed and accessibility of transit stops has increased magnificently.  

Given the success of this project in Barcelona, we have created a similar model for the city of San Diego, where our analyses of the foot-traffic in various locations all over the city gives the ideal position of where a ‘Superblock’ can be placed to improve commercial activity and community engagement as well as reduce pollution. We obtained an index to represent how ‘busy' different locations are in San Diego, using three different data-sets from different local businesses, transit stations and parking meters that comprehensively cover foot-traffic of a given area, compared using geographical coordinates. After gathering this information, we recognized certain locations in San Diego that can be cordoned off to create a healthier and more efficient society, which we have geo-spatially illustrated in this project. 

##### Referencess:
  ######  1)https://www.theguardian.com/cities/2016/may/17/superblocks-rescue-barcelona-spain-plan-give-streets-back-residents
   ###### 2) https://www.walkscore.com/professional/research.php 
   ###### 3)https://www.wired.com/2017/04/brilliant-simplicity-new-yorks-new-times-square/ 
   ###### 4)http://krqe.com/2017/03/06/city-councilor-wants-wider-sidewalks-to-help-businesses-impacted-by-art/
   ###### 5)http://www.nyc.gov/html/dot/downloads/pdf/dot-economic-benefits-of-sustainable-streets.pdf

## Data Description

'Business.csv' is an offical San Diego govt dataset that contains the list of all active registered businesses in San Diego as of April 2017. 

'FY2014 Ridership_Trolley_Sept2013Booking.csv' is an official San Diego govt dataset that gives us the number of people that use transit system in San Deigo as of April 2017. 

'treas_parking_payments_2017_datasd.csv' is an official San Deigo govt dataset that gives us the information of all the parking meter transactions upto April 2017. 

## Data Cleaning / Pre-processing

In [3]:
#Importing the original file for active business data in SD

fname = "Business.csv"
business_df = pd.read_csv(fname)

IOError: File Business.csv does not exist

In [ ]:
#Importing the clean file for train stops data in SD

fname1 = "Stop_Counts.csv"
stops_df = pd.read_csv(fname1)

In [ ]:
#Importing the clean file for parking meter transactions data in SD

fname2 = "Parking_Counts.csv"
parking_df = pd.read_csv(fname2)

### Active Businesses 

In [ ]:
#Removing unneccesary columns and renaming to layman terms, filtering all business into places that serve food/drinks

business_df = business_df[['doing_bus_as_name','zip','naics_description','lat','lon']]
business_df = business_df.loc[(business_df['naics_description'] == 'full-service restaurants') |
               (business_df['naics_description'] == 'cafeterias') | 
               (business_df['naics_description'] == 'food services & drinking places') |
               (business_df['naics_description'] == 'limited-service eating places') |
               (business_df['naics_description'] == 'limited-service restaurants')  |
               (business_df['naics_description'] == 'mobile food services') |
               (business_df['naics_description'] == 'drinking places (alcoholic beverages)') |
               (business_df['naics_description'] == 'snack & nonalcoholic beverage bars')]
business_df.rename(columns = {'doing_bus_as_name':'Business title','naics_description':'Type of Place'}, inplace=True)
business_df = business_df.reset_index(drop=True)

In [ ]:
#Adding the 5 columns for the different time brackets

business_df.insert(2,'AM early','Null')
business_df.insert(3,'AM peak','Null')
business_df.insert(4,'Mid-day','Null')
business_df.insert(5,'PM peak','Null')
business_df.insert(6,'PM late','Null')

In [ ]:
#generalising the different categories into 3 categories: Only Food, Food & Drinks, Only Drinks

business_df.loc[(business_df['Type of Place'] == 'mobile food services') |
                (business_df['Type of Place'] == 'cafeterias') |
                (business_df['Type of Place'] == 'snack & nonalcoholic beverage bars') |
                (business_df['Type of Place'] == 'limited-service eating places') |
                (business_df['Type of Place'] == 'limited-service restaurants') 
                , 'Type of Place'] = 'Only Food'

business_df.loc[(business_df['Type of Place'] == 'full-service restaurants') 
                , 'Type of Place'] = 'Food & Drinks'

business_df.loc[(business_df['Type of Place'] == 'drinking places (alcoholic beverages)') |
                (business_df['Type of Place'] == 'food services & drinking places') 
                , 'Type of Place'] = 'Only Drinks'

In [ ]:
#Assigning values to different time brackets by assuming foot traffic according to the type of place 

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('3','25','20','18','5')
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('7','28','25','35','25')
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('27','6','3','27','30') 

In [ ]:
#Assigning a total score to all three categories (summ of scores of all time brackets)

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                'Total Score'] = '71'
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                'Total Score'] = '120'
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                'Total Score'] = '93'

In [ ]:
business_df

In [ ]:
business_df.to_csv('Restaurant_Counts.csv')

### Transit Stops Dataset
The following cells take the original dataset and clean it so that we get the cleaned data, NOT MEANT TO BE RUN since we already have the cleaned dataset

#Code which we used to clean the original dataset which is way too big for Github 

trips = pd.read_csv('FY2014 Ridership_Trolley_Sept2013Booking.csv')

# Removing unnecessary columns: all we need is stop_id, passengers getting off on the station 
# and time_arrival to get the time split for the number of people getting off at a station at a time_period

trips = trips[['STOP_ID', 'PASSENGERS_OFF', 'TIME_ACTUAL_ARRIVE']]
trips.columns = ['stop_id', 'count', 'time']
trips

##### clean the data, removing any na rows we will also remove all rows that have value 0

trips.dropna(how='any')
trips = trips[trips['count'] != 0]
trips

# Using stop_id we can connect with lat/long. We will be grouping by pole_id to create a table with 
# the following columns: (stop_id, latitude, longitude, count_am_early, count_am_peak, count_midday, 
# count_pm_early, count_pm_late, count_daily)

#### Getting location data for all the stops using another dataset
stop_locs = pd.read_csv('FY2014 Ridership_Trolley_Sept2013_Stops.csv')
stop_locs = stop_locs[['STOP_ID', 'LAT', 'LON']]
stop_locs

stop_locs.columns = ['stop_id', 'latitude', 'longitude']

stops_df = trips.merge(stop_locs, how='left')
stops_df = stops_df.dropna(how='any')
stops_df

##### grouping by stop_id and aggregating over the count values

stop_counts = stops_df.groupby(['stop_id']).agg('count')

##### remove unnecessary columns

stop_counts = stop_counts['time'] 
stop_counts = stop_counts.to_frame()
stop_counts['stop_id'] = stop_counts.index
stop_counts = stop_counts.merge(stop_locs, how='left')
stop_counts.columns = ['total_count', 'stop_id', 'latitude', 'longitude']
stop_counts

##### We need to now find the number of days in the transactions dataset
##### We will be using this in order to get the count of transactions PER DAY

stops_df['time'] = pd.to_datetime(stops_df['time'])
dates = stops_df['time']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

##### The time ranges for which we have split the transit ridership data are:
AM_Early = 12AM-6AM
AM_Late = 6AM-9AM
..
..


#### Classify the time slot based on the times during the day
def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
stops_df['time_slot'] = stops_df['time'].apply(classify)
stops_df

##### Merging the two datasets by key value and column name

def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val

### Setting the values of the transactions for each time period during the day
def set_temporal_counts(p_id):
    v_counts = stops_df.loc[stops_df['stop_id'] == p_id]['time_slot'].value_counts(dropna=False)
    stop_counts.loc[stop_counts['stop_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']

stop_counts['stop_id'].apply(set_temporal_counts)

stop_counts.to_csv('Stop_Counts.csv')

In [ ]:
#cleaned dataset for the transit spots
stops_df

### Parking meters 

###### Importing data and initial cleaning

In [ ]:
#importing original dataset
parking = pd.read_csv('treas_parking_payments_2017_datasd.csv')

In [ ]:
#removing unnecessary columns: all we need is pole_id, time_start and meter_expire
parking = parking.drop(['uuid'], axis=1)
parking = parking.drop(['trans_amt'], axis=1)
parking = parking.drop(['pay_method'], axis=1)
parking = parking.drop(['meter_type'], axis=1)
parking = parking.drop(['meter_expire'], axis=1)

In [ ]:
#clean the data, removing any na rows 
parking.dropna(how='any')

###### Connecting with other dataset (with lat/long pairs)

In [ ]:
# 2.) using pole_id we can connect with lat/long. We will be grouping by pole_id to create
#     a table with the following columns:
#     (pole_id, latitude, longitude, count_am_early, count_am_peak, 
#        count_midday, count_pm_early, count_pm_late, count_daily)
# NOTE: we will need to decide whether to use raw numbers or averages of counts per section per day

In [ ]:
park_loc = pd.read_csv('treas_parking_meters_loc_datasd.csv')

In [ ]:
park_loc = park_loc[['pole', 'longitude', 'latitude']]
park_loc.columns = ['pole_id', 'longitude', 'latitude']
park_loc = park_loc.dropna(how='any')

In [ ]:
park_df = parking.merge(park_loc, how='left')
park_df = park_df.dropna(how='any')
park_df

In [ ]:
park_counts = park_df.groupby(['pole_id']).agg('count')
park_counts = park_counts['trans_start'] #remove unnecessary columns
park_counts = park_counts.to_frame()
park_counts['pole_id'] = park_counts.index
park_counts = park_counts.merge(park_loc, how='left')
park_counts.columns = ['total_count', 'pole_id', 'longitude', 'latitude']
park_counts

In [ ]:
# we need to now find the number of days in the transactions dataset
# we will be using this in order to get the count of transactions PER DAY
park_df['trans_start'] = pd.to_datetime(park_df['trans_start'])
dates = park_df['trans_start']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
park_df['time_slot'] = park_df['trans_start'].apply(classify)
park_df

In [ ]:
def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val
def set_temporal_counts(p_id):
    v_counts = park_df.loc[park_df['pole_id'] == p_id]['time_slot'].value_counts(dropna=False)
    park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
#     park_counts.loc[park_counts['pole_id'] == ]
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']


park_counts['pole_id'].apply(set_temporal_counts)




# v_counts = park_df.loc[park_df['pole_id'] == 'N-1003']['time_slot'].value_counts(dropna=False)
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
# park_counts.loc[park_counts['pole_id'] == 'N-1003']



In [ ]:
max_date = dates.max()
min_date = dates.min()
days_elapsed = (max_date - min_date).days + 1 #to round off this number
days_elapsed

In [4]:
park_counts.to_csv("Parking_Counts.csv");

NameError: name 'park_counts' is not defined

In [ ]:
# let's now get average counts of parked vehicles
# dividing by total days to give per daily expected counts 
park_counts['total_count'] = park_counts['total_count'].apply(lambda x: x / days_elapsed)
park_counts['am_early'] = park_counts['am_early'].apply(lambda x: x / days_elapsed)
park_counts['am_peak'] = park_counts['am_peak'].apply(lambda x: x / days_elapsed)
park_counts['midday'] = park_counts['midday'].apply(lambda x: x / days_elapsed)
park_counts['pm_peak'] = park_counts['pm_peak'].apply(lambda x: x / days_elapsed)
park_counts['pm_late'] = park_counts['pm_late'].apply(lambda x: x / days_elapsed)

In [ ]:
# we then multiply this amount by the mean number of people per vehicle
# as per https://www.rita.dot.gov/bts/sites/rita.dot.gov.bts/files/publications/highlights_of_the_2001_national_household_travel_survey/html/table_a15.html
ppl_per_vehicle = 1.63
park_counts['total_count'] = park_counts['total_count'].apply(lambda x: x * ppl_per_vehicle)
park_counts['am_early'] = park_counts['am_early'].apply(lambda x: x * ppl_per_vehicle)
park_counts['am_peak'] = park_counts['am_peak'].apply(lambda x: x * ppl_per_vehicle)
park_counts['midday'] = park_counts['midday'].apply(lambda x:  x * ppl_per_vehicle)
park_counts['pm_peak'] = park_counts['pm_peak'].apply(lambda x:  x * ppl_per_vehicle)
park_counts['pm_late'] = park_counts['pm_late'].apply(lambda x:  x * ppl_per_vehicle)
park_counts

In [ ]:
park_counts = pd.read_csv("Parking_Counts.csv");
####Rough Calculations for algorithm
long_max = park_counts['longitude'].max()
long_min = park_counts['longitude'].min()
lat_max = park_counts['latitude'].max()
lat_min = park_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    park_counts.loc[park_counts['pole_id'] == s,'row'] =  (park_counts.loc[park_counts['pole_id'] == s,'latitude'] - lat_min) // lat_gap
    park_counts.loc[park_counts['pole_id'] == s,'col'] = (park_counts.loc[park_counts['pole_id'] == s,'longitude'] - long_min) // long_gap

park_counts['pole_id'].apply(classify_blocks)

park_counts

In [ ]:
print(long_max)
print(long_min)
print(long_dif)
print(lat_max)
print(lat_min)
print(lat_dif)
park_counts = park_counts.loc[park_counts['longitude'] != -180.0 ] #remove the outlier
long_min = park_counts['longitude'].min()
long_min

In [ ]:
park_counts.to_csv("Parking_Counts_grid.csv");

## Data Analysis and Results 

In [ ]:
park_counts = pd.read_csv('Parking_Counts.csv')
park_counts = park_counts.loc[park_counts['longitude'] != -180.0 ] #remove the outlier
park_counts.rename(columns={'pole_id':'id'}, inplace= True)
park_counts['type'] = 'parking'
del park_counts['Unnamed: 0']
park_counts 

In [ ]:
restaurants = pd.read_csv('Restaurant_Counts.csv')
restaurants.columns = ['id', 'title', 'zip', 'am_early', 'am_peak', 'midday', 'pm_peak', 'pm_late', 'type', 'latitude', 'longitude', 'total_count']
restaurants['total_count'] = restaurants['total_count'].apply( lambda x : x * 2/3)
restaurants['am_peak'] = restaurants['am_peak'].apply( lambda x : x * 2/3)
restaurants['am_early'] = restaurants['am_early'].apply( lambda x : x * 2/3)
restaurants['midday'] = restaurants['midday'].apply( lambda x : x * 2/3)
restaurants['pm_peak'] = restaurants['pm_peak'].apply( lambda x : x * 2/3)
restaurants['pm_late'] = restaurants['pm_late'].apply( lambda x : x * 2/3)
restaurants['id'] = restaurants['id'].apply(lambda x : ('R-' + str(x) ))
restaurants = restaurants[['id','type','title', 'am_early', 'am_peak', 'midday', 'pm_peak', 'pm_late', 'total_count','latitude', 'longitude']]

restaurants

In [ ]:
transit = pd.read_csv('Stop_Counts.csv')
transit.rename(columns={'stop_id':'id'}, inplace= True)
transit['type'] = 'transit_stop'
del transit['Unnamed: 0'] 
transit

In [ ]:
location_counts = restaurants.append(park_counts).append(transit)
# Setting a new index
#location_counts.index = idx # new ad hoc index
# location_counts.index = range(len(location_counts)) # set with list
# location_counts = location_counts.reset_index() # replace old w new
# location_counts.rename(columns={'index':'idx'}, inplace= True)
# location_counts.rename(columns={'id':'type_id'}, inplace= True)
# del location_counts['Unnamed: 0']
location_counts

In [ ]:
long_max = location_counts['longitude'].max()
long_min = location_counts['longitude'].min()
lat_max = location_counts['latitude'].max()
lat_min = location_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    location_counts.loc[location_counts['id'] == s,'row'] =  (location_counts.loc[location_counts['id'] == s,'latitude'] - lat_min) // lat_gap
    location_counts.loc[location_counts['id'] == s,'col'] = (location_counts.loc[location_counts['id'] == s,'longitude'] - long_min) // long_gap

location_counts['id'].apply(classify_blocks)

location_counts

In [ ]:
print(location_counts['row'].min())
print(location_counts['row'].max())
print(location_counts['row'].mean())
print(location_counts['row'].std())
print(location_counts['col'].min())
print(location_counts['col'].max())
print(location_counts['col'].mean())
print(location_counts['col'].std())

In [ ]:
grid = []
for i in range(NUMBER_BLOCKS_ROOT):
    for j in range(NUMBER_BLOCKS_ROOT):
        grid.append([i, j, (lat_min+ i*lat_gap), (long_min + j*long_gap)])    
grid = pd.DataFrame(grid)
grid.columns = ['row','col','lat','lon']
grid
#grid.loc[(grid['row']==142) & (grid['col']==378)]

In [ ]:
location_counts

In [ ]:
location_counts.to_csv('Location_grid.csv')

In [ ]:
total_loc_counts = location_counts[['total_count','row','col']]
amearly_loc_counts = location_counts[['am_early','row','col']]
ampeak_loc_counts = location_counts[['am_peak','row','col']]
midday_loc_counts = location_counts[['midday','row','col']]
pmlate_loc_counts = location_counts[['pm_late','row','col']]
pmpeak_loc_counts = location_counts[['pm_peak','row','col']]
total_loc_counts

In [ ]:
weighted_grid = total_loc_counts.groupby(['row','col']).sum().sort_values(by='total_count',ascending=False)
weighted_grid2 = amearly_loc_counts.groupby(['row','col']).sum().sort_values(by='am_early',ascending=False)
weighted_grid3 = ampeak_loc_counts.groupby(['row','col']).sum().sort_values(by='am_peak',ascending=False)
weighted_grid4 = midday_loc_counts.groupby(['row','col']).sum().sort_values(by='midday',ascending=False)
weighted_grid5 = pmlate_loc_counts.groupby(['row','col']).sum().sort_values(by='pm_late',ascending=False)
weighted_grid6 = pmpeak_loc_counts.groupby(['row','col']).sum().sort_values(by='pm_peak',ascending=False)
weighted_grid

In [ ]:
# merge these two dataframes where the values in grid 
#weighted_grid = pd.merge(weighted_grid, grid, on=['row')
big_grid = pd.merge(grid, 
                weighted_grid.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid.fillna(0.0)

big_grid2 = pd.merge(grid, 
                weighted_grid2.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid2.fillna(0.0)

big_grid3 = pd.merge(grid, 
                weighted_grid3.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid3.fillna(0.0)

big_grid4 = pd.merge(grid, 
                weighted_grid4.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid4.fillna(0.0)

big_grid5 = pd.merge(grid, 
                weighted_grid5.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid5.fillna(0.0)

big_grid6 = pd.merge(grid, 
                weighted_grid6.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid6.fillna(0.0)

In [ ]:
big_grid['total_count'].max()

In [ ]:
big_grid2['am_early'].max()

In [ ]:
big_grid3['am_peak'].max()

In [ ]:
big_grid4['midday'].max()

In [ ]:
big_grid5['pm_late'].max()

In [ ]:
big_grid6['pm_peak'].max()

In [ ]:
big_grid

In [ ]:
big_grid2

In [ ]:
big_grid3

In [ ]:
big_grid4

In [ ]:
big_grid5

In [ ]:
big_grid6

In [ ]:
big_grid.to_csv('Heat_Map1.csv')
big_grid2.to_csv('Heat_Map_am_early.csv')
big_grid3.to_csv('Heat_Map_am_peak.csv')
big_grid4.to_csv('Heat_Map_midday.csv')
big_grid5.to_csv('Heat_Map_pm_late.csv')
big_grid6.to_csv('Heat_Map_pm_peak.csv')


## Data Visualization

In [ ]:
grid = pd.read_csv('Heat_Map.csv')
grid2 = pd.read_csv('AM_Early_Heat_Map.csv')
grid3 = pd.read_csv('AM_Peak_Heat_Map.csv')
grid4 = pd.read_csv('Midday_Heat_Map.csv')
grid5 = pd.read_csv('PM_Late_Heat_Map.csv')
grid6 = pd.read_csv('PM_Peak_Heat_Map.csv')

In [ ]:
#print values as strings or empty str if they are null
def null_print(val):
    if(val is not None):
        return str(val)
    else:
        return ''

#get_addresses
def get_address(loc):
    geo = geocoder.google(loc, method='reverse')
    if(geo is not None):
        address = null_print(geo.housenumber) + ' ' + null_print(geo.street) + ' ' + null_print(geo.city) + ' ' + null_print(geo.postal)
    else: 
        address = "Unknown"
    return address


In [ ]:
weight_feature = 'total_count'#change this for different timeslots


#stats to use for classification
mean = grid[weight_feature].astype(float).mean()
std = grid[weight_feature].astype(float).std()

#convert to floats for folium to read these
grid['lat'] = grid['lat'].astype(float)
grid['lon'] = grid['lon'].astype(float)

#color constants for viz
COLOR_LOW = '#50f442'
COLOR_MILD = '#fff375'
COLOR_MEDIUM = '#ffa154'
COLOR_HIGH = '#ff5454'
COLOR_MAX = '#a43dff'

#depending on the weight we assign graded colors
def weight_to_color(w):
    if(w < mean - std):
        return COLOR_LOW
    elif (w < mean):
        return COLOR_MILD
    elif (w < mean + std):
        return COLOR_MEDIUM
    elif (w < (mean + 2 * std)):
        return COLOR_HIGH
    else:
        return COLOR_MAX


for index, row in grid.iterrows():
    lat = np.asscalar(row['lat'])
    lon = np.asscalar(row['lon'])
    loc = [lat,lon]
    if(not math.isnan(lon) and not math.isnan(lat)):
        weight = np.asscalar(row[weight_feature])
        color = weight_to_color(weight)
        address = get_address(loc)
        folium.RegularPolygonMarker(loc, popup=address, fill_color=color, rotation = -45, number_of_sides=4, radius=10, fill_opacity = 0.66).add_to(map_sd)

map_sd.save("total_count.html")


In [ ]:
weight_feature = 'am_early'#change this for different timeslots


#stats to use for classification
mean = grid2[weight_feature].astype(float).mean()
std = grid2[weight_feature].astype(float).std()

#convert to floats for folium to read these
grid2['lat'] = grid2['lat'].astype(float)
grid2['lon'] = grid2['lon'].astype(float)

#color constants for viz
COLOR_LOW = '#50f442'
COLOR_MILD = '#fff375'
COLOR_MEDIUM = '#ffa154'
COLOR_HIGH = '#ff5454'
COLOR_MAX = '#a43dff'

#depending on the weight we assign graded colors
def weight_to_color(w):
    if(w < mean - std):
        return COLOR_LOW
    elif (w < mean):
        return COLOR_MILD
    elif (w < mean + std):
        return COLOR_MEDIUM
    elif (w < (mean + 2 * std)):
        return COLOR_HIGH
    else:
        return COLOR_MAX


for index, row in grid2.iterrows():
    lat = np.asscalar(row['lat'])
    lon = np.asscalar(row['lon'])
    loc = [lat,lon]
    if(not math.isnan(lon) and not math.isnan(lat)):
        weight = np.asscalar(row[weight_feature])
        color = weight_to_color(weight)
        address = get_address(loc)
        folium.RegularPolygonMarker(loc, popup=address, fill_color=color, rotation = -45, number_of_sides=4, radius=10, fill_opacity = 0.66).add_to(map_sd2)

map_sd2.save("am_early.html")


In [ ]:
weight_feature = 'am_peak'#change this for different timeslots


#stats to use for classification
mean = grid3[weight_feature].astype(float).mean()
std = grid3[weight_feature].astype(float).std()

#convert to floats for folium to read these
grid3['lat'] = grid3['lat'].astype(float)
grid3['lon'] = grid3['lon'].astype(float)

#color constants for viz
COLOR_LOW = '#50f442'
COLOR_MILD = '#fff375'
COLOR_MEDIUM = '#ffa154'
COLOR_HIGH = '#ff5454'
COLOR_MAX = '#a43dff'

#depending on the weight we assign graded colors
def weight_to_color(w):
    if(w < mean - std):
        return COLOR_LOW
    elif (w < mean):
        return COLOR_MILD
    elif (w < mean + std):
        return COLOR_MEDIUM
    elif (w < (mean + 2 * std)):
        return COLOR_HIGH
    else:
        return COLOR_MAX


for index, row in grid3.iterrows():
    lat = np.asscalar(row['lat'])
    lon = np.asscalar(row['lon'])
    loc = [lat,lon]
    if(not math.isnan(lon) and not math.isnan(lat)):
        weight = np.asscalar(row[weight_feature])
        color = weight_to_color(weight)
        address = get_address(loc)
        folium.RegularPolygonMarker(loc, popup=address, fill_color=color, rotation = -45, number_of_sides=4, radius=10, fill_opacity = 0.66).add_to(map_sd3)

map_sd3.save("am_peak.html")


In [ ]:
weight_feature = 'midday'#change this for different timeslots


#stats to use for classification
mean = grid4[weight_feature].astype(float).mean()
std = grid4[weight_feature].astype(float).std()

#convert to floats for folium to read these
grid4['lat'] = grid4['lat'].astype(float)
grid4['lon'] = grid4['lon'].astype(float)

#color constants for viz
COLOR_LOW = '#50f442'
COLOR_MILD = '#fff375'
COLOR_MEDIUM = '#ffa154'
COLOR_HIGH = '#ff5454'
COLOR_MAX = '#a43dff'

#depending on the weight we assign graded colors
def weight_to_color(w):
    if(w < mean - std):
        return COLOR_LOW
    elif (w < mean):
        return COLOR_MILD
    elif (w < mean + std):
        return COLOR_MEDIUM
    elif (w < (mean + 2 * std)):
        return COLOR_HIGH
    else:
        return COLOR_MAX


for index, row in grid4.iterrows():
    lat = np.asscalar(row['lat'])
    lon = np.asscalar(row['lon'])
    loc = [lat,lon]
    if(not math.isnan(lon) and not math.isnan(lat)):
        weight = np.asscalar(row[weight_feature])
        color = weight_to_color(weight)
        address = get_address(loc)
        folium.RegularPolygonMarker(loc, popup=address, fill_color=color, rotation = -45, number_of_sides=4, radius=10, fill_opacity = 0.66).add_to(map_sd4)

map_sd4.save("midday.html")


In [ ]:
weight_feature = 'pm_late'#change this for different timeslots


#stats to use for classification
mean = grid5[weight_feature].astype(float).mean()
std = grid5[weight_feature].astype(float).std()

#convert to floats for folium to read these
grid5['lat'] = grid5['lat'].astype(float)
grid5['lon'] = grid5['lon'].astype(float)

#color constants for viz
COLOR_LOW = '#50f442'
COLOR_MILD = '#fff375'
COLOR_MEDIUM = '#ffa154'
COLOR_HIGH = '#ff5454'
COLOR_MAX = '#a43dff'

#depending on the weight we assign graded colors
def weight_to_color(w):
    if(w < mean - std):
        return COLOR_LOW
    elif (w < mean):
        return COLOR_MILD
    elif (w < mean + std):
        return COLOR_MEDIUM
    elif (w < (mean + 2 * std)):
        return COLOR_HIGH
    else:
        return COLOR_MAX


for index, row in grid5.iterrows():
    lat = np.asscalar(row['lat'])
    lon = np.asscalar(row['lon'])
    loc = [lat,lon]
    if(not math.isnan(lon) and not math.isnan(lat)):
        weight = np.asscalar(row[weight_feature])
        color = weight_to_color(weight)
        address = get_address(loc)
        folium.RegularPolygonMarker(loc, popup=address, fill_color=color, rotation = -45, number_of_sides=4, radius=10, fill_opacity = 0.66).add_to(map_sd5)

map_sd5.save("pm_late.html")


In [ ]:
weight_feature = 'pm_peak'#change this for different timeslots


#stats to use for classification
mean = grid6[weight_feature].astype(float).mean()
std = grid6[weight_feature].astype(float).std()

#convert to floats for folium to read these
grid6['lat'] = grid6['lat'].astype(float)
grid6['lon'] = grid6['lon'].astype(float)

#color constants for viz
COLOR_LOW = '#50f442'
COLOR_MILD = '#fff375'
COLOR_MEDIUM = '#ffa154'
COLOR_HIGH = '#ff5454'
COLOR_MAX = '#a43dff'

#depending on the weight we assign graded colors
def weight_to_color(w):
    if(w < mean - std):
        return COLOR_LOW
    elif (w < mean):
        return COLOR_MILD
    elif (w < mean + std):
        return COLOR_MEDIUM
    elif (w < (mean + 2 * std)):
        return COLOR_HIGH
    else:
        return COLOR_MAX


for index, row in grid6.iterrows():
    lat = np.asscalar(row['lat'])
    lon = np.asscalar(row['lon'])
    loc = [lat,lon]
    if(not math.isnan(lon) and not math.isnan(lat)):
        weight = np.asscalar(row[weight_feature])
        color = weight_to_color(weight)
        address = get_address(loc)
        folium.RegularPolygonMarker(loc, popup=address, fill_color=color, rotation = -45, number_of_sides=4, radius=10, fill_opacity = 0.66).add_to(map_sd6)

map_sd6.save("pm_peak.html")


In [ ]:
"""
Purple = Very High
Red = High
Orange = Medium
Yellow = Low
"""

map_sd #Map for Total counts

In [ ]:
map_sd2 #Map for am early

In [ ]:
map_sd3 #Map for am peak

In [ ]:
map_sd4 #Map for midday

In [ ]:
map_sd5 #Map for pm late

In [ ]:
map_sd6 #Map for pm peak

## Conclusions/Discussion